## Imports

In [12]:
import urllib, json
import pandas as pd

# HTML parsing
from lxml import html
import requests

## Getting all the Spanish movies

Source: Wikipedia. https://es.wikipedia.org/wiki/Categor%C3%ADa:Pel%C3%ADculas_de_Espa%C3%B1a

Since there was no existing database with Spanish films and their scores in the Bechdel test, it was necessary to create one of our own.
To do so, we have opted to scrape the information of all the Spanish production (or co-production) films that we could find (source: Wikipedia).
In addition, we have made use of the API offered by https://bechdeltest.com/ (the API documentation can be accessed via the following link: https://bechdeltest.com/api/v1/doc). Thanks to the method "getMovieByImdbId" it has been possible to obtain the Bechdel score of 172 films (out of the 3 149 that were scraped from Wikipedia).
To do this, it was necessary to obtain the IMDb ID of each of the films. This was achieved by means of XPath expressions.

<img src="img/wikipedia_imdbid.png" width=1000 height=800/>

In [ ]:
wikipedia_spanish_movies_imdbid_list = []

# AQUÍ HAY QUE METER UNA LISTA CON LOS LINKS DE LAS DISTINTAS PÁGINAS DE 200 EN 200 PELÍCULAS
# O IR SCRAPEANDO EL ENLACE DE LA SIGUENTE PÁGINA HASTA LA ÚLTIMA

for i in range(1):
    wikipedia_page = requests.get('https://es.wikipedia.org/w/index.php?title=Categor%C3%ADa:Pel%C3%ADculas_de_Espa%C3%B1a&pagefrom=Un+verano+para+matar#mw-pages')
    wikipedia_tree = html.fromstring(wikipedia_page.content)
    wikipedia_links_list = wikipedia_tree.xpath('//div[@class="mw-category-group"]/ul/li/a[@href]')
    wikipedia_links_href_list = [f'https://es.wikipedia.org/{link.get("href")}' for link in wikipedia_links_list]
    for movie_link in wikipedia_links_href_list:
        movie_page = requests.get(movie_link)
        movie_tree = html.fromstring(movie_page.content)
        movie_imdbid = movie_tree.xpath('//td[@class="navbox-list navbox-odd"]/div/ul/li[5]/span[@class="uid"]/a/text()')
        if movie_imdbid:
            movie_imdbid = movie_imdbid[0][2:]
            print(movie_imdbid)
            wikipedia_spanish_movies_imdbid_list.append(movie_imdbid)
        else:
            print('List is empty')

## Getting Spanish movies that have Bechdel Score alredy rated

As previously mentioned, once the IMDb IDs of all the films produced in Spain have been obtained, requests are made to the Bechdel test API.
As expected, as these are Spanish films, not all of them have been tested and collected in this database, so we have to deal with these situations in which the API returns an empty list.
When the movie is in the database, the API returns a JSON, as shown in the following image.

<img src="img/json_example_bechdel.png" width=300 height=300/>

Once the list with the JSON objects is obtained, we convert it to a data frame.
The reason why we add all the elements to a list first and then convert it to a data frame instead of adding each JSON object as a new row of a data frame is because the runtime is much longer than adding elements to a list. This can be seen in the following graph.

<img src="img/runtime_list_df_append.png" width=600 height=600/>

In [ ]:
df_wikipedia_spanish_movies_list = []
for wikipedia_spanish_movie_imdbid in wikipedia_spanish_movies_imdbid_list:
    http_request = f'http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid={wikipedia_spanish_movie_imdbid}'
    new_wikipedia_movie = pd.read_json(http_request, typ="series")
    df_wikipedia_spanish_movies_list.append(new_wikipedia_movie)

df_wikipedia_spanish_movies = pd.DataFrame(df_wikipedia_spanish_movies_list)

The final data frame will also contain films that could not be found in the Bechdel test database. It is easy to discard these rows, because in the column "description" they contain the string "Could not find movie".

In [ ]:
df_wikipedia_spanish_movies = df_wikipedia_spanish_movies.drop(df_wikipedia_spanish_movies[df_wikipedia_spanish_movies.description == "Could not find movie"].index)
df_wikipedia_spanish_movies.reset_index(inplace=True)

Finally, the data frame information is saved in a CSV file.

In [ ]:
df_wikipedia_spanish_movies.to_csv("wikipedia_spanish_movies_bechdel_new.csv", mode='a', columns=["index", "status", "version", "description", "date", "dubious", "year", "visible", "rating", "title", "submitterid", "id", "imdbid"], header=False, index=False)

## Getting the IMDb information of Spanish movies

Because one of the analyses to be carried out needs to know the rating of each film, it is necessary to scrape this information, this time from the IMDb website. The f-string is used to generate the URLs of each film's page in IMDb, which will be scraped to obtain the rating.

<img src="img/url_rating_imdb.png" width=1000 height=800/>

<img src="img/html_rating_imdb.png" width=1000 height=800/>

<img align="left" src="img/genre_example.png" width=630 height=600/>
<img align="right" src="img/director_example.png" width=630 height=600/>

<img align="left" src="img/budget_example.png" width=630 height=600/>
<img align="right" src="img/gross_example.png" width=630 height=600/>

As before, the information we need to perform the various analyses must be scraped by means of XPath expressions.

Always keep in mind that it is not possible to assume that all pages will have the same HTML structure or even contain the HTML element you are looking for. This is why it is necessary to build XPath expressions that are resistant to these variations and always check for possible cases where the result of scraping is an empty list.

In [40]:
df_spanish_movies_bechdel = pd.read_csv("def_wikipedia_spanish_movies_bechdel.csv", dtype='str')

In [41]:
imdb_movies_list = []
imdbid_list = df_spanish_movies_bechdel["imdbid"].to_list()
for imdbid in imdbid_list:
    imdb_movie_page = requests.get(f'https://www.imdb.com/title/tt{imdbid}/?ref_=fn_tt_tt_1')
    imdb_movie_tree = html.fromstring(imdb_movie_page.content)
    
    # As mentioned above, XPath expressions are built to be resistant to variations and the response 
    # is always checked to ensure that it is not empty.
    
    rating_movie = imdb_movie_tree.xpath('(//span[@class="sc-7ab21ed2-1 jGRxWM"])[1]/text()')
    rating = rating_movie[0] if rating_movie else "No rating"
    
    genre_movie = imdb_movie_tree.xpath('(//span[@class="ipc-chip__text"])[1]/text()')
    genre = genre_movie[0] if genre_movie else "No genre"
    
    director_movie = imdb_movie_tree.xpath('(//a[@class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"])[1]/text()')
    director = director_movie[0] if director_movie else "No director"
    
    budget_movie = imdb_movie_tree.xpath('//span[text()="Budget"]/following-sibling::div[1]/ul/li/span/text()')
    budget = budget_movie[0].split(" ")[0][1:] if budget_movie else "No budget"
    
    gross_movie = imdb_movie_tree.xpath('//span[text()="Gross worldwide"]/following-sibling::div[1]/ul/li/span/text()')
    gross = gross_movie[0].split(" ")[0][1:] if gross_movie else "No gross"
    
    # Once all the necessary data has been obtained, a dictionary is compiled, which will be added to a list 
    # that will eventually contain the information on all the films.
    # As mentioned in previous sections, these elements are added to a list instead of a data frame because it is 
    # much faster and later it will be enough to convert the list to a data frame.
    
    new_movie = {"imdbid": imdbid,
                "rating": rating,
                "genre": genre,
                "director": director,
                "budget": budget,
                "gross": gross}
    
    imdb_movies_list.append(new_movie)

imdb_movies_df = pd.DataFrame(imdb_movies_list)

In [42]:
imdb_movies_df

,imdbid,rating,genre,director,budget,gross
0,0088461,7.1,Comedy,Pedro Almodóvar,No budget,No gross
1,0809533,6.6,Drama,Emilio Martínez Lázaro,"9,000,000","6,939,516"
2,0421528,6.6,Comedy,Ramón Salazar,No budget,"439,632"
3,1600383,6.9,Drama,Jon Garaño,No budget,"115,251"
4,0306432,6.2,Comedy,Daniela Fejerman,No budget,"2,447,070"
...,...,...,...,...,...,...
118,5862312,6.2,Horror,Paco Plaza,No budget,"6,123,089"
119,2896036,7.0,Comedy,David Trueba,No budget,"3,075,169"
120,1847746,6.7,Drama,Lucía Puenzo,"2,000,000","3,635,718"
121,0995829,7.0,Drama,Lucía Puenzo,No budget,"2,728,869"


In [46]:
# Commas are removed from numbers to avoid problems with writing and reading data in CSV format.
imdb_movies_df = imdb_movies_df.apply(lambda x: x.str.replace(',',''))

In [48]:
imdb_movies_df.to_csv("spanish_movies_data.csv", index=False)